In [1]:
import numpy as np
import pandas as pd
import pytz
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import MetaTrader5 as mt5
import json

In [2]:
# connect to MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# request connection status and parameters
print(mt5.terminal_info())
# get data on MetaTrader 5 version
print(mt5.version())

# set time zone to UTC
timezone = pytz.timezone("Etc/UTC")
# create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
utc_from = datetime(2020, 1, 10, tzinfo=timezone)
utc_to = datetime(2020, 1, 11, tzinfo=timezone)

# set time zone to UTC
timezone = pytz.timezone("Etc/UTC")

TerminalInfo(community_account=True, community_connection=True, connected=True, dlls_allowed=True, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=True, build=4410, maxbars=100000, codepage=1252, ping_last=46110, community_balance=0.0, retransmission=7.003783969728242, company='MetaQuotes Software Corp.', name='MetaTrader 5', language='Spanish', path='C:\\Program Files\\MetaTrader 5', data_path='C:\\Users\\iamfr\\AppData\\Roaming\\MetaQuotes\\Terminal\\D0E8209F77C8CF37AD8BF550E51FF075', commondata_path='C:\\Users\\iamfr\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')
(500, 4410, '21 Jun 2024')


In [3]:
# Import stock data form mt5

# create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
utc_from = datetime(2024, 1, 1, tzinfo=timezone)
utc_to = datetime(2024, 7, 16, tzinfo=timezone)
# request AUDUSD ticks within 11.01.2020 - 11.01.2020
ticks = mt5.copy_ticks_range("GC_Q", utc_from, utc_to, mt5.COPY_TICKS_ALL)
print("Ticks received:",len(ticks))

mt5.shutdown()

Ticks received: 7768136


True

In [4]:
# create DataFrame out of the obtained data
df = pd.DataFrame(ticks)
# convert time in seconds into the datetime format
df['time']=pd.to_datetime(df['time'], unit='s')

df = df.set_index('time')
del df['last']
del df['flags']
del df['volume']
del df['time_msc']
del df['volume_real']

# display data
print("\nDisplay dataframe with ticks")
print(df.head(10)) 


Display dataframe with ticks
                        bid     ask
time                               
2024-01-02 01:00:05  2129.0  2129.5
2024-01-02 01:00:06  2128.9  2129.5
2024-01-02 01:00:07  2128.9  2129.4
2024-01-02 01:00:10  2128.6  2129.1
2024-01-02 01:00:13  2128.6  2129.2
2024-01-02 01:00:13  2128.6  2129.1
2024-01-02 01:00:15  2128.6  2129.2
2024-01-02 01:00:16  2128.5  2129.0
2024-01-02 01:00:17  2128.5  2129.1
2024-01-02 01:00:17  2128.5  2129.0


In [5]:
# Create an iteration for analise each day aperture
zones_extracted = []
flag_full_session = True

for index1, day in df.groupby(df.index.date):
    
    # Iterating each session
    start_session = pd.to_datetime(index1.strftime('%Y-%m-%d') + ' ' + '15:30:00')
    end_session = pd.to_datetime(index1.strftime('%Y-%m-%d') + ' ' + '16:00:00')

    # DF contains the session to analise.
    if flag_full_session:
        refdf = day
    else:
        refdf = day.loc[start_session:end_session]
    
    # create a MinMaxScaler object
    scaler = MinMaxScaler()

    # fit and transform the data
    normalized_data = scaler.fit_transform(refdf)

    # create a new DataFrame with the normalized data
    ndf = pd.DataFrame(normalized_data, columns=refdf.columns)

    ndf['ask'] = ndf['ask'].fillna(ndf['bid'])
    ndf['bid'] = ndf['bid'].fillna(ndf['ask'])

    ndf_to_matrix = ndf.values
    # Preparing data for clustering: Normalize time and price to have similar scales
    X_time = np.linspace(0, 1, len(ndf_to_matrix)).reshape(-1, 1)
    X_price = (ndf['ask'].values - np.min(ndf['ask'])) / (np.max(ndf['ask']) - np.min(ndf['ask']))
    X_cluster = np.column_stack((X_time, X_price))

    # Applying KMeans clustering
    num_clusters = 3
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(ndf_to_matrix)

    # Extract cluster centers and rescale back to original price range
    cluster_centers = kmeans.cluster_centers_[:, 1] * (np.max(refdf['ask']) - np.min(refdf['ask'])) + np.min(refdf['ask'])
    
    zones = cluster_centers.tolist()
    output = {
        "date": index1.strftime('%Y-%m-%d'),
        "zones": zones,
    }
    zones_extracted.append(output)

print(zones_extracted)   

[{'date': '2024-01-02', 'zones': [2132.5776847517964, 2140.095013486914, 2125.871835803877]}, {'date': '2024-01-03', 'zones': [2108.899689440994, 2099.6242063645877, 2125.745119367951]}, {'date': '2024-01-04', 'zones': [2111.1809492706293, 2104.417889864018, 2107.8812286142]}, {'date': '2024-01-05', 'zones': [2092.8247384287997, 2121.654053493676, 2107.159385383912]}, {'date': '2024-01-08', 'zones': [2104.377372785912, 2092.961972037186, 2084.243093214623]}, {'date': '2024-01-09', 'zones': [2096.342077787519, 2099.9501118113435, 2091.1877506179617]}, {'date': '2024-01-10', 'zones': [2096.862341463415, 2086.7480950484537, 2092.1055745478607]}, {'date': '2024-01-11', 'zones': [2088.344382877434, 2077.060140503471, 2093.0757482640274]}, {'date': '2024-01-12', 'zones': [2095.8113022390166, 2116.3531833796055, 2107.020047477745]}, {'date': '2024-01-15', 'zones': [2115.651997371879, 2113.2563260903066, 2108.350932902763]}, {'date': '2024-01-16', 'zones': [2087.7434052757794, 2109.77011916308

In [6]:
print(len(zones_extracted))

with open('zones_extracted_.json', 'w') as file:
    json.dump(zones_extracted, file)

139
